In [550]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import os
import webbrowser
import time
import json
import numpy as np

In [191]:
pd.set_option("display.max_columns", 30)

In [321]:
projects = pd.read_excel("Projects_v3.xlsx", sheet_name="VCM BC projects")
# i don't concat directly because I wanted to harmonize the column names
sheets = {'car':'CAR Projects','vcs': 'VCS Projects', 'acr': 'ACR Projects','gold': 'Gold Projects'}
berkeley = {}
for v, k in sheets.items():
    berkeley[v] = pd.read_excel("Voluntary-Registry-Offsets-Database--v8-May-2023.xlsx", sheet_name=k).dropna(how='all', axis=1)

berkeley['vcs'] = berkeley['vcs'].drop(['Project ID'], axis=1)
berkeley['vcs'] = berkeley['vcs'].rename(columns={"Registry ID": "Project ID"})
berkeley['gold'] = berkeley['gold'].rename(columns={"GS_ID": "Project ID"})
berkeleyDS = pd.concat(berkeley.values())
berkeleyDS.dropna(subset=['Project ID'],inplace=True)

In [291]:
# only VCS has mangrove projects in the project names
additional = berkeley['vcs'][berkeley['vcs']['Project Name'].str.contains(r'mangrove', na=False, case=False)]

In [314]:
projects['registryID'] = projects['registryID'].astype(str)
projects.loc[projects['Registry entry'].str.contains(r'thereserve2', na=False), 'projectID'] = 'CAR' + projects['registryID'] 
projects.loc[projects['Registry entry'].str.contains(r'verra', na=False), 'projectID'] = 'VCS' + projects['registryID'] 

In [324]:
# drop duplicated columns as we merge
projectsM = projects.merge(berkeleyDS, left_on="projectID",right_on="Project ID", how="left", suffixes=('', '_y'))
projectsM.dropna(how='all', axis=1, inplace=True)
projectsM.drop(projectsM.filter(regex='_y$').columns, axis=1, inplace=True)
projectsM.reset_index(drop=True, inplace=True)

In [355]:
# redundant, but in case you use for the Berkeley dataset
projectsM.loc[projectsM['projectID'].str.contains(r'CAR', na=False), 'projectLink'] = 'https://thereserve2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1=' + projectsM['registryID'] 
projectsM.loc[projectsM['projectID'].str.contains(r'VCS', na=False), 'projectLink'] = 'https://registry.verra.org/uiapi/resource/resourceSummary/' + projectsM['registryID'] 

In [580]:
%%time
projectsM['urlDown'] = ''
projectsM['urlFileNames'] = ''
for url in ['https://registry.verra.org/uiapi/resource/resourceSummary/1463']:
    if type(url) is str:
        # VCS and CAR for now
        if 'verra' in url:
            data = requests.get(url).json()
            # we get the lon,lat as in the JSON file
            projectsM.loc[projectsM['projectLink'] == url, 'latitude'] = data['location']['latitude']
            projectsM.loc[projectsM['projectLink'] == url, 'longitude'] = data['location']['longitude']
            # we also get the KML, if any, to compute the coordinates manually
            # the nesting in the JSON file is nasty
            found_match = False
            for group in data['documentGroups']:
                if 'OTHER_DOCUMENTS' in next(iter(group.values())):
                    for docs in group['documents']:
                        regexp = re.compile(r'\.kml',re.IGNORECASE)
                        if regexp.search(docs['documentName']):
                            projectsM.loc[projectsM['projectLink'] == url, 'urlDown'] = docs['uri']
                            projectsM.loc[projectsM['projectLink'] == url, 'urlFileNames'] = docs['documentName']
                            found_match = True
                            break  # Stop looking for more matches
                if found_match:
                    break  # Stop iterating over 'documentGroups'
        elif 'thereserve2' in url:
            reqs = requests.get(url)
            soup = BeautifulSoup(reqs.text, 'html.parser')
            links = soup.find_all('a', href=True, string=re.compile(r'shapefile|shape file|shape|shp|kml', re.IGNORECASE))
            if len(links)>0:
                projectsM.loc[projectsM['projectLink'] == url, 'urlDown'] = 'https://thereserve2.apx.com' + links[0].get('href')
                projectsM.loc[projectsM['projectLink'] == url, 'urlFileNames'] = links[0].text

CPU times: user 29 ms, sys: 7.92 ms, total: 36.9 ms
Wall time: 773 ms


In [585]:
# check what files have been downloaded already 
projectsM.replace(r'^\s*$', np.nan, regex=True, inplace=True)
downloaded = [
    os.path.splitext(filename)[0] for filename in os.listdir("./locationFiles/")
]
downList = []
for project in list(set(projectsM['projectID'])):
        if project not in downloaded:
                downList.append(project)

In [587]:
# download the file and name it with as the project ID. 
for url in projectsM.urlDown:
      if type(url) is str:
        if projectsM['projectID'][projectsM['urlDown'] == url].astype(str).values[0] in downList:
            try:
                 downFile = requests.get(url)
                 fileName = './locationFiles/{}'.format(projectsM['projectID'][projectsM['urlDown'] == url].astype(str).values[0])
                 ext = re.findall('(\.[^.]*)$',projectsM['urlFileNames'][projectsM['urlDown'] == url].astype(str).values[0])[-1]
                 if ext != 'zip':  
                      with open("%s%s" % (fileName, ext), 'wb') as f:
                                for chunk in downFile.iter_content(1024): # iterate on stream using 1KB packets
                                    f.write(chunk) # write the file
                 else:
                      with open(fileName, 'wb') as f:
                                for chunk in downFile.iter_content(1024): # iterate on stream using 1KB packets
                                    f.write(chunk) # write the file               
            #   open('./locationFiles/{}'.format(projectsM['projectID'][projectsM['urlDown'] == url].astype(str).values[0]), 'wb').write(downFile.content)
            except:
                 pass
            time.sleep(2)

## Trash

In [290]:
# it will find nonsense values for non-registry ones, but we filter later
# projects["registryID"] = (
#     projects["Registry entry"].apply(
#         lambda x: re.findall(r"-?\d+\.?\d*", x)[-1] if type(x) == str else x
#     )
# ).astype("Int64")

# berkeleyDS['registryID'] = (
#     berkeleyDS["Project ID"].apply(
#         lambda x: re.findall(r"-?\d+\.?\d*", x)[-1] if type(x) == str else x
#     )
# ).astype("Int64")
